In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
import sqlite3

In [3]:
# Connect to SQLite DB (or create if not exists)
conn = sqlite3.connect('student_management.db')
c = conn.cursor()

In [5]:
# Create tables
c.execute('''CREATE TABLE IF NOT EXISTS students (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                age INTEGER,
                grade TEXT
            )''')

c.execute('''CREATE TABLE IF NOT EXISTS attendance (
                student_id INTEGER,
                date TEXT,
                status TEXT,
                PRIMARY KEY(student_id, date),
                FOREIGN KEY(student_id) REFERENCES students(id)
            )''')

conn.commit()

In [7]:
# Main app window
root = tk.Tk()
root.title("Student Management System")
root.geometry("800x550")

''

In [9]:
# ---------------------------- FUNCTIONS ---------------------------- #

def load_students():
    for row in tree.get_children():
        tree.delete(row)
    c.execute("SELECT * FROM students")
    for row in c.fetchall():
        tree.insert("", tk.END, values=row)

def add_student():
    name = entry_name.get()
    age = entry_age.get()
    grade = entry_grade.get()

    if not name or not age or not grade:
        messagebox.showerror("Input Error", "Please fill all fields.")
        return

    try:
        c.execute("INSERT INTO students (name, age, grade) VALUES (?, ?, ?)", (name, int(age), grade))
        conn.commit()
        messagebox.showinfo("Success", f"Student '{name}' added.")
        entry_name.delete(0, tk.END)
        entry_age.delete(0, tk.END)
        entry_grade.delete(0, tk.END)
        load_students()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def mark_attendance():
    selected = tree.focus()
    if not selected:
        messagebox.showerror("Select Student", "Please select a student from the list.")
        return

    student = tree.item(selected)['values']
    student_id = student[0]
    date = entry_date.get()
    status = combo_status.get()

    if not date or status == "":
        messagebox.showerror("Input Error", "Please enter date and select attendance status.")
        return

    try:
        c.execute("INSERT OR REPLACE INTO attendance (student_id, date, status) VALUES (?, ?, ?)",
                  (student_id, date, status))
        conn.commit()
        messagebox.showinfo("Success", "Attendance recorded.")
    except Exception as e:
        messagebox.showerror("Error", str(e))

def view_grades():
    selected = tree.focus()
    if not selected:
        messagebox.showerror("Select Student", "Please select a student from the list.")
        return

    student = tree.item(selected)['values']
    messagebox.showinfo("Grade", f"Student: {student[1]}\nGrade: {student[3]}")

In [11]:
# ---------------------------- UI ELEMENTS ---------------------------- #

# Frame: Add Student
frame_add = tk.LabelFrame(root, text="Add Student")
frame_add.pack(fill="x", padx=10, pady=5)

tk.Label(frame_add, text="Name:").grid(row=0, column=0, padx=5, pady=5)
entry_name = tk.Entry(frame_add)
entry_name.grid(row=0, column=1, padx=5, pady=5)

tk.Label(frame_add, text="Age:").grid(row=0, column=2, padx=5, pady=5)
entry_age = tk.Entry(frame_add)
entry_age.grid(row=0, column=3, padx=5, pady=5)

tk.Label(frame_add, text="Grade:").grid(row=0, column=4, padx=5, pady=5)
entry_grade = tk.Entry(frame_add)
entry_grade.grid(row=0, column=5, padx=5, pady=5)

btn_add = tk.Button(frame_add, text="Add Student", command=add_student)
btn_add.grid(row=0, column=6, padx=10)

In [14]:
# Frame: Student List
frame_list = tk.LabelFrame(root, text="Student List")
frame_list.pack(fill="both", expand=True, padx=10, pady=5)

columns = ("ID", "Name", "Age", "Grade")
tree = ttk.Treeview(frame_list, columns=columns, show='headings')
for col in columns:
    tree.heading(col, text=col)
tree.pack(fill="both", expand=True)

In [16]:
# Frame: Attendance
frame_attendance = tk.LabelFrame(root, text="Mark Attendance")
frame_attendance.pack(fill="x", padx=10, pady=5)

tk.Label(frame_attendance, text="Date (YYYY-MM-DD):").grid(row=0, column=0, padx=5, pady=5)
entry_date = tk.Entry(frame_attendance)
entry_date.grid(row=0, column=1, padx=5, pady=5)

tk.Label(frame_attendance, text="Status:").grid(row=0, column=2, padx=5, pady=5)
combo_status = ttk.Combobox(frame_attendance, values=["Present", "Absent"])
combo_status.grid(row=0, column=3, padx=5, pady=5)

btn_attendance = tk.Button(frame_attendance, text="Mark Attendance", command=mark_attendance)
btn_attendance.grid(row=0, column=4, padx=10)

In [18]:
# Button: View Grade
btn_grade = tk.Button(root, text="View Selected Student Grade", command=view_grades)
btn_grade.pack(pady=5)

In [20]:
# Load initial student list
load_students()

In [ ]:
# Run the app
root.mainloop()